# Week3
## DCOPF 建模
根据《数据驱动最优潮流的误差可信量化及其抑制方法》式 2.2 至 2.5，DCOPF 具体建模如下

\begin{aligned}
\min_{P_G}\quad & C_G^{\!\top} P_G \qquad\text{(linear objective 线性目标)} \\ \text{s.t.}\quad & \mathbf{1}_G^{\!\top} P_G = \mathbf{1}_D^{\!\top} P_D \quad\text{(system balance 系统功率平衡)} \\ & P_L^{\min} \ \le\ M_G P_G + M_D P_D \ \le\ P_L^{\max} \quad\text{(line limits 线路约束)} \\ & P_G^{\min} \ \le\ P_G \ \le\ P_G^{\max} \quad\text{(generator bounds 机组约束).}
\end{aligned}
 

In [1]:
# pyright: reportAttributeAccessIssue=false, reportIndexIssue=false, reportOperatorIssue=false, reportGeneralTypeIssues=false, reportCallIssue=false, reportArgumentType=false, reportOptionalOperand=false, reportOptionalSubscript=false
import pyomo.environ as pyo
from typing import Any

def build_dc_opf_model() -> Any:
    """Abstract DC-OPF model (variables: P_G)
    Obj: minimize generation cost
    Constraints: system balance, line limits and generator bounds.
    """
    m: Any = pyo.AbstractModel()

    # --------- Sets ---------
    m.G = pyo.Set(doc="Generators")
    m.D = pyo.Set(doc="Demands")
    m.L = pyo.Set(doc="Lines") 

    # --------- Parameters ---------
    # Linear generation cost coefficients
    m.c2 = pyo.Param(m.G, within=pyo.Reals, doc="Linear cost coefficient C_G")
    m.c1 = pyo.Param(m.G, within=pyo.Reals, doc="Linear cost coefficient")
    m.c0 = pyo.Param(m.G, within=pyo.Reals, doc="Constant cost coefficient")

    # Demand vector (indexed by m.D)
    m.Pd = pyo.Param(m.D, doc="Demand at each demand node (fixed)")

    # Line flow limits
    m.P_L_max = pyo.Param(m.L, within=pyo.Reals, doc="Line flow upper limit")
    m.P_L_min = pyo.Param(m.L, within=pyo.Reals, doc="Line flow lower limit")

    # PTDFs: contribution of each generator / demand to line flows
    m.M_G = pyo.Param(m.L, m.G, within=pyo.Reals, doc="PTDFs for generators")
    m.M_D = pyo.Param(m.L, m.D, within=pyo.Reals, doc="PTDFs for demands")

    # Generator output bounds
    m.P_G_max = pyo.Param(m.G, within=pyo.Reals, doc="Generator max output")
    m.P_G_min = pyo.Param(m.G, within=pyo.Reals, doc="Generator min output")

    # --------- Variables ---------
    def _P_G_bounds(m: Any, g):
        return (m.P_G_min[g], m.P_G_max[g])
    m.P_G = pyo.Var(m.G, bounds=_P_G_bounds, doc="Generator outputs (MW)")

    # --------- Objective ---------
    def _gen_cost_rule(m: Any):
        return sum(m.c2[g] * m.P_G[g]**2 + m.c1[g] * m.P_G[g] + m.c0[g] for g in m.G)
    m.Obj = pyo.Objective(rule=_gen_cost_rule, sense=pyo.minimize)

    # --------- Constraints ---------
    # System balance: sum_g P_G = sum_d P_d
    def _system_balance_rule(m: Any):
        return sum(m.P_G[g] for g in m.G) == sum(m.Pd[d] for d in m.D)
    m.System_Balance = pyo.Constraint(rule=_system_balance_rule)

     # Line flow expression: f_l = M_G*P_G + M_D*P_d
    def _flow_def(m: Any, l):
        return sum(m.M_G[l, g]*m.P_G[g] for g in m.G) \
             + sum(m.M_D[l, d]*m.Pd[d] for d in m.D)
    m.Flow = pyo.Expression(m.L, rule=_flow_def)

    # Line limits: P_L_min <= f_l <= P_L_max
    def _flow_limits(m: Any, l):
        return pyo.inequality(m.P_L_min[l], m.Flow[l], m.P_L_max[l])
    m.LineLimits = pyo.Constraint(m.L, rule=_flow_limits)

    return m


In [2]:
# pyright: reportAttributeAccessIssue=false, reportIndexIssue=false, reportOperatorIssue=false, reportGeneralTypeIssues=false, reportCallIssue=false, reportArgumentType=false, reportOptionalOperand=false, reportOptionalSubscript=false
import numpy as np
import pyomo.environ as pyo

# ---- helpers to build PTDF & Pyomo data from a PYPOWER case ----

def _bus_index_maps(ppc):
    bus = ppc["bus"]
    # PYPOWER uses 1-based bus numbers in the data; build map -> 0-based row indices
    bus_nums = bus[:, 0].astype(int)
    idx_of_busnum = {int(bn): i for i, bn in enumerate(bus_nums)}
    return bus_nums, idx_of_busnum

def _choose_slack(ppc, idx_of_busnum):
    # type column is 2nd field (index 1): 3=REF, 2=PV, 1=PQ
    REF = 3
    bus = ppc["bus"]
    ref_rows = np.where(bus[:,1] == REF)[0]
    if len(ref_rows) == 0:
        # fallback: smallest bus number
        return 0
    return int(ref_rows[0])

def _build_dc_matrices(ppc, slack_row):
    """Return (Bbus, Cf, b_line) with tap ratios accounted for, angles ignored."""
    bus = ppc["bus"]
    branch = ppc["branch"]

    nb = bus.shape[0]
    nl = branch.shape[0]

    # columns in branch: fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
    FBUS, TBUS, R, X, B, RATEA, RATEB, RATEC, RATIO, ANGLE, STATUS, ANGMIN, ANGMAX = range(13)

    # active lines
    on = branch[:, STATUS] > 0.5
    br = branch[on, :]

    # susceptance per branch (p.u. on system base); DC uses 1/x, include tap ratio
    x = br[:, X]
    if np.any(np.isclose(x, 0.0)):
        raise ValueError("Branch reactance x=0 found; DC PTDF undefined for that branch.")
    tap = br[:, RATIO].copy()
    tap[ (tap==0) | np.isnan(tap) ] = 1.0  # default tap=1
    b_series = 1.0 / x
    b_line = b_series / tap  # effective susceptance on "from" side

    # incidence matrix Cf (nl_on x nb): +1 at from, -1 at to
    fbus = br[:, FBUS].astype(int)
    tbus = br[:, TBUS].astype(int)

    # map bus numbers -> row indices
    bus_nums, idx_map = _bus_index_maps(ppc)
    nl_on = br.shape[0]
    Cf = np.zeros((nl_on, nb))
    for k in range(nl_on):
        i = idx_map[int(fbus[k])]
        j = idx_map[int(tbus[k])]
        Cf[k, i] =  1.0 / tap[k]   # tap on from-side
        Cf[k, j] = -1.0

    # Bf and Bbus
    Bf   = (b_line[:, None] * Cf)                   # f = Bf * theta
    Bbus = Cf.T @ (b_line[:, None] * Cf)            # p = Bbus * theta

    # Reduce by removing slack row/col for inversion
    keep = [i for i in range(nb) if i != slack_row]
    Bbus_red = Bbus[np.ix_(keep, keep)]

    return Bbus, Bbus_red, Bf, Cf, b_line, keep

def _compute_ptdf(ppc):
    """Compute PTDF (lines × buses) w.r.t. chosen slack."""
    _, idx_map = _bus_index_maps(ppc)
    slack_row = _choose_slack(ppc, idx_map)
    Bbus, Bbus_red, Bf, Cf, _, keep = _build_dc_matrices(ppc, slack_row)

    # PTDF via columns: for each bus k, inject +1 at k and -1 at slack
    nb = Bbus.shape[0]
    nl = Bf.shape[0]
    H = np.zeros((nl, nb))  # PTDF[:, bus]

    # Pre-factorize reduced Bbus for speed
    # (Cholesky may fail if singular; use np.linalg.solve)
    for k in range(nb):
        if k == slack_row:
            # by definition, a 1 p.u. injection at slack balanced at slack gives zero net — column zeros
            H[:, k] = 0.0
            continue
        # build p_red: +1 at k, -1 at slack -> in reduced system it's +1 at k, slack removed
        p = np.zeros(nb)
        p[k] =  1.0
        p[slack_row] = -1.0
        p_red = p[keep]
        theta_red = np.linalg.solve(Bbus_red, p_red)

        # reconstruct full theta (insert 0 at slack position)
        theta = np.zeros(nb)
        theta[keep] = theta_red
        theta[slack_row] = 0.0

        # line flows for that unit transaction
        f = Bf @ theta
        H[:, k] = f

    # Only rows for in-service branches were built; map them back to full list of lines if needed
    return H, slack_row

def ppc_to_pyomo_data_for_create_instance(ppc):
    """Return (data_dict, meta) where data_dict matches Pyomo create_instance(data=...)."""
    baseMVA = ppc["baseMVA"]
    bus = ppc["bus"]; gen = ppc["gen"]; branch = ppc["branch"]; gencost = ppc["gencost"]

    bus_nums, idx_map = _bus_index_maps(ppc)
    nb = bus.shape[0]; ng = gen.shape[0]; nl = branch.shape[0]

    # --- Sets (string labels) ---
    G = [f"g{g}" for g in range(ng)]
    PD_COL = 2  # Pd col in bus
    load_rows = [i for i in range(nb) if bus[i, PD_COL] > 1e-9]
    D = [f"d{bus_nums[i]}" for i in load_rows]
    L = [f"l{k}" for k in range(nl)]

    # --- Gen bounds & mapping gen->bus row ---
    PMAX, PMIN, GBUS = 8, 9, 0
    Pg_max = {G[g]: float(gen[g, PMAX]) for g in range(ng)}
    Pg_min = {G[g]: float(gen[g, PMIN]) for g in range(ng)}
    g_bus_rows = [idx_map[int(gen[g, GBUS])] for g in range(ng)]

    # --- Quadratic costs (type 2) ---
    c2 = {}; c1 = {}; c0 = {}
    for g in range(ng):
        c2[G[g]] = float(gencost[g, 4])
        c1[G[g]] = float(gencost[g, 5])
        c0[G[g]] = float(gencost[g, 6])

    # --- Demands ---
    Pd = {f"d{bus_nums[i]}": float(bus[i, PD_COL]) for i in load_rows}

    # --- Line limits ---
    RATEA, STATUS = 5, 10
    bigM = 1e6
    P_L_max = {}
    P_L_min = {}
    active_mask = branch[:, STATUS] > 0.5
    for k in range(nl):
        lim = float(branch[k, RATEA]) if (active_mask[k] and branch[k, RATEA] > 0) else bigM
        P_L_max[f"l{k}"] = lim
        P_L_min[f"l{k}"] = -lim

    # --- PTDF (lines_in_service × buses) ---
    H, slack_row = _compute_ptdf(ppc)

    # Map PTDF back to all lines (zeros for out-of-service)
    on = np.where(active_mask)[0]
    H_full = np.zeros((nl, nb))
    H_full[on, :] = H

    # --- Build M_G and M_D ---
    M_G = {}
    for g in range(ng):
        col = H_full[:, g_bus_rows[g]]
        for k in range(nl):
            M_G[(f"l{k}", G[g])] = float(col[k])

    M_D = {}
    for i in load_rows:
        dkey = f"d{bus_nums[i]}"
        col = H_full[:, i]
        for k in range(nl):
            M_D[(f"l{k}", dkey)] = float(-col[k])  # minus for loads

    # --- Pack in Pyomo's required shapes ---
    data_dict = {
        # Sets must be wrapped as {None: iterable}
        "G": {None: list(G)},
        "D": {None: list(D)},
        "L": {None: list(L)},

        # 1-D Params (keys match model component names)
        "P_G_min": Pg_min,
        "P_G_max": Pg_max,
        "c2": c2, "c1": c1, "c0": c0,
        "Pd": Pd,
        "P_L_min": P_L_min,
        "P_L_max": P_L_max,

        # 2-D Params: keys are tuples of indices used by the model
        "M_G": M_G,
        "M_D": M_D,
    }

    inner = {
        "G": {None: list(G)},
        "D": {None: list(D)},
        "L": {None: list(L)},
        "P_G_min": Pg_min,
        "P_G_max": Pg_max,
        "c2": c2, "c1": c1, "c0": c0,
        "Pd": Pd,
        "P_L_min": P_L_min, "P_L_max": P_L_max,
        "M_G": M_G,
        "M_D": M_D,
    }
    data_wrapped = {None: inner}
    meta = {"slack_row": int(slack_row), "baseMVA": float(baseMVA)}
    return data_wrapped, meta


In [3]:
# pyright: reportOptionalOperand=false, reportCallIssue=false, reportArgumentType=false, reportOptionalSubscript=false
from typing import Optional

# Load case and solve DCOPF
from pypower.api import case118

ppc = case118()
model = build_dc_opf_model()

data_dict, meta = ppc_to_pyomo_data_for_create_instance(ppc)
instance = model.create_instance(data=data_dict)   # now OK

opt = pyo.SolverFactory('ipopt')
res = opt.solve(instance, tee=False)


print('Solver termination:', res.solver.termination_condition)
print('Objective value: ${:,.2f}/hr'.format(pyo.value(instance.Obj)))

# Display generator dispatch results
print(f'\nGenerator dispatch (showing active generators):')
print(f'{"Generator":<12} {"Output (MW)":>12}')
print('-' * 26)

active_count = 0
for g in sorted(list(instance.G), key=lambda x: int(str(x)[1:])):
    Pg_val_raw: Optional[float] = pyo.value(instance.P_G[g])
    if Pg_val_raw is None:
        continue
    Pg_val: float = float(Pg_val_raw)
    # Only show generators with non-trivial output
    if Pg_val > 0.01:
        print(f'{g:<12} {Pg_val:12.2f}')
        active_count += 1

print('-' * 26)
nG = len(list(instance.G))
total_generation: float = sum(float(pyo.value(instance.P_G[g]) or 0.0) for g in instance.G)
print(f'Active generators: {active_count} of {nG}')
print(f'Total generation: {total_generation:,.2f} MW')


Solver termination: optimal
Objective value: $125,947.87/hr

Generator dispatch (showing active generators):
Generator     Output (MW)
--------------------------
g4                 436.08
g5                  82.37
g10                213.20
g11                304.29
g13                  6.78
g19                 18.41
g20                197.69
g21                 46.52
g24                150.21
g25                155.05
g27                378.91
g28                379.87
g29                500.43
g36                462.24
g38                  3.88
g39                588.22
g44                244.21
g45                 38.76
g50                 34.89
--------------------------
Active generators: 19 of 54
Total generation: 4,242.00 MW


In [4]:
# pyright: reportOptionalSubscript=false, reportArgumentType=false
from typing import Any, Iterable

# --- Run DCOPF using PYPOWER built-in solver -----------------
from pypower.api import case118, rundcopf

# Load the case
ppc = case118()

# Run DC-OPF
results = rundcopf(ppc)

# ---- Print generator outputs ----
print("\nGenerator dispatch results (MW):")
# Safely access generator matrix from results without ambiguous truth checks
gens_val = results.get("gen") if isinstance(results, dict) else None
if gens_val is None:
    gens_iter: Iterable[Any] = []
else:
    gens_iter = gens_val

# Column 0 = bus number, 1 = Pg, 8 = Pmax in MATPOWER/PYPOWER format
for i, row in enumerate(gens_iter, start=1):
    bus_num = int(row[0])
    Pg = row[1]
    Pmax = row[8]
    print(f"  Gen {i:02d} at Bus {bus_num:3d}:  Pg = {Pg:10.4f} MW   (Pmax = {Pmax:10.4f})")

PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.04 seconds
Objective Function Value = 125947.87 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses            118     Total Gen Capacity    9966.2           0.0 to 0.0
Generators        54     On-line Capacity      9966.2           0.0 to 0.0
Committed Gens    54     Generation (actual)   4242.0               0.0
Loads             99     Load                  4242.0               0.0
  Fixed           99       Fixed               4242.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches         186     Losses (I^2 * Z)         0

### Extract $P_G^{min}$ and $P_G^{max}$ from `.xlsx` File

In [5]:
import pandas as pd
from pathlib import Path

def load_generator_limits(xlsx_path, sheet_index=2):
    """
    Read generator limits from the third sheet of an Excel file.

    Parameters
    ----------
    xlsx_path : str
        Path to your .xlsx file
    sheet_index : int
        Sheet index (0-based). The 3rd sheet → 2.

    Returns
    -------
    gen_bus : ndarray of shape (ng,)
        Bus numbers of generators (1-based IDs)
    PGmax : ndarray of shape (ng,)
        Generator upper bounds (p.u.)
    PGmin : ndarray of shape (ng,)
        Generator lower bounds (p.u.)
    """

    df = pd.read_excel(xlsx_path, sheet_name=sheet_index)

    # Normalize column names to be safe
    df.columns = [c.strip() for c in df.columns]

    # Expected column names:
    # 'Gen_Bus', 'Pmax(p.u.)', 'Pmin(p.u.)'
    gen_bus = df['Gen_Bus'].to_numpy(dtype=int)
    PGmax   = df['Pmax(p.u.)'].to_numpy(dtype=float)
    PGmin   = df['Pmin(p.u.)'].to_numpy(dtype=float)

    print(f"Loaded {len(gen_bus)} generator entries from {xlsx_path}")
    return gen_bus, PGmax, PGmin

### Extracte and Preprocess Data for Concrete Model

In [11]:
# pyright: reportAttributeAccessIssue=false, reportArgumentType=false, reportOperatorIssue=false
import numpy as np
from pypower.api import case118
ppc = case118()

# Extract set G
ng = ppc['gen'].shape[0]  # number of generators
G = np.arange(ng)
"""
gen_bus = ppc['gen'][:, 0].astype(int) - 1  # 0-based bus indices
m.bus_of_gen = pyo.Param(m.G, initialize=lambda m,i: int(gen_bus[i]))
"""

# Extract cost coefficients
gencost = ppc['gencost']
c2 = gencost[:, 4]
c1 = gencost[:, 5]
c0 = gencost[:, 6]

# Extract Set D
nb = ppc["bus"].shape[0]  # number of buses (demands)
D = np.arange(nb)

# Extract Set L
branch = ppc['branch']
nbr = branch.shape[0] # number of branches (lines) sometimes nl in comments
L = np.arange(nbr)

# Extract P_L^{min} and P_L^{max}
PLmax = np.full(nbr, 9900.0, dtype=float)     # shape (nbr,)
PLmin = np.full(nbr, -9900.0, dtype=float)    # shape (nbr,)

# Extract P_G^{min} and P_G^{max}
xlsx_path = Path.cwd() / "IEEE_118_bus_test_system.xlsx"  # no leading slash
gen_bus, PGmax_pu, PGmin_pu = load_generator_limits(xlsx_path)
# Convert from p.u. to MW: multiply by baseMVA (= 100.0 for case118)
baseMVA = float(ppc["baseMVA"])
PGmax = PGmax_pu * baseMVA  # MW
PGmin = PGmin_pu * baseMVA  # MW
print("Gen bus (first 5):", gen_bus[4:7])
print("PGmax (MW, first 5):", PGmax[4:7])
print("PGmin (MW, first 5):", PGmin[4:7])

# Extract M_G and M_D
FBUS, TBUS, R, X, B, RATEA, RATEB, RATEC, RATIO, ANGLE, STATUS = range(11)
# 1) tap ratios (taps) and series reactances (X)
t = branch[:, RATIO].copy()
t[t == 0] = 1.0                         # defualt tap = 1
x = branch[:, X].copy()
# 2) active-line mask; we’ll build on active rows then expand back
active = (branch[:, STATUS] > 0) & (x != 0.0)
idx_act = np.where(active)[0]
nl_act = idx_act.size
# 3) tap-weighted incidence C_f (active rows only)
Cf = np.zeros((nl_act, nb))
for r, ell in enumerate(idx_act):
    i = int(branch[ell, FBUS]) - 1      # 0-based
    j = int(branch[ell, TBUS]) - 1
    Cf[r, i] =  1.0 / t[ell]
    Cf[r, j] = -1.0
# 4) DC “line couplings”
b = 1.0 / (x[idx_act] * t[idx_act])     # b_ell = 1/(x_ell * t_ell)
Bf = (b[:, None]) * Cf                  # diag(b) @ Cf
# 5) Nodal susceptance and slack reduction
Bbus = Cf.T @ ((b[:, None]) * Cf)       # Cf^T diag(b) Cf  (nb x nb)
slack = 0                               # choose a slack bus (0-based)
keep = [k for k in range(nb) if k != slack]
Bbus_red = Bbus[np.ix_(keep, keep)]
inv_Bbus_red = np.linalg.inv(Bbus_red)
E = np.eye(nb)[:, keep]                 # (nb x (nb-1))
# This constructs the “pinv” that enforces theta_slack = 0
Bbus_pinv = E @ inv_Bbus_red @ E.T      # (nb x nb)
# 6) PTDF for active lines; insert zero rows for out-of-service
H_act = Bf @ Bbus_pinv                  # (nl_act x nb)
H_full = np.zeros((nbr, nb))
H_full[idx_act, :] = H_act              # zeros for inactive lines
# 7) Map to MG, MD
gen_bus_0b = ppc['gen'][:, 0].astype(int) - 1    # length ng
nb_d = nb                                        # we index demands by bus
MG = H_full[:, gen_bus_0b]                       # (nl x ng)
MD = -H_full                                     # (nl x nb) : demand at bus d is -injection

Loaded 54 generator entries from e:\DOCUMENT\Learn_Py\opf\Week3\IEEE_118_bus_test_system.xlsx
Gen bus (first 5): [10 12 15]
PGmax (MW, first 5): [550. 185. 100.]
PGmin (MW, first 5): [0. 0. 0.]


### Concrete Model with Data Embedded

In [ ]:
# pyright: reportAttributeAccessIssue=false, reportArgumentType=false, reportOperatorIssue=false
import pyomo.environ as pyo
import numpy as np

# ============================================================================
# Build Concrete DCOPF Model from Cell 1 Formula using Cell 9 Data
# ============================================================================
# Formula from Cell 1:
# min_{P_G}  C_G^T P_G  (quadratic: sum c2*PG^2 + c1*PG + c0)
# s.t.  1_G^T P_G = 1_D^T P_D         (system balance)
#       P_L^min <= M_G P_G + M_D P_D <= P_L^max  (line limits)
#       P_G^min <= P_G <= P_G^max     (generator bounds)
# ============================================================================

# All data from Cell 9: G, D, L, c2, c1, c0, PGmin, PGmax, PLmin, PLmax, MG, MD
# Dimensions:
#   G: np.arange(ng), D: np.arange(nb), L: np.arange(nl)
#   c2, c1, c0, PGmin, PGmax: shape (ng,) in MW
#   PLmin, PLmax: shape (nl,) in MW
#   MG: shape (nl, ng), MD: shape (nl, nb) - PTDFs (dimensionless)
#   PD: shape (nb,) in MW - will load from ppc

ng = len(G)
nb = len(D)
nl = len(L)

# Load demand from case118 (MW)
baseMVA = float(ppc['baseMVA'])
PD_MW = ppc['bus'][:, 2]  # column 2 is Pd in MW

print(f"Building Concrete DCOPF Model:")
print(f"  Generators: {ng}")
print(f"  Buses (demands): {nb}")
print(f"  Lines: {nl}")
print(f"  Total demand: {PD_MW.sum():.2f} MW")

# Build Pyomo ConcreteModel
m = pyo.ConcreteModel(name="DCOPF_Concrete")

# --- Sets ---
m.G = pyo.Set(initialize=range(ng), doc="Generator indices")
m.D = pyo.Set(initialize=range(nb), doc="Bus/demand indices")
m.L = pyo.Set(initialize=range(nl), doc="Line indices")

# --- Parameters (all in MW or dimensionless) ---
m.c2 = pyo.Param(m.G, initialize={g: float(c2[g]) for g in range(ng)}, doc="Quadratic cost coeff")
m.c1 = pyo.Param(m.G, initialize={g: float(c1[g]) for g in range(ng)}, doc="Linear cost coeff")
m.c0 = pyo.Param(m.G, initialize={g: float(c0[g]) for g in range(ng)}, doc="Constant cost term")

m.PGmin = pyo.Param(m.G, initialize={g: float(PGmin[g]) for g in range(ng)}, doc="Gen min (MW)")
m.PGmax = pyo.Param(m.G, initialize={g: float(PGmax[g]) for g in range(ng)}, doc="Gen max (MW)")

m.PLmin = pyo.Param(m.L, initialize={l: float(PLmin[l]) for l in range(nl)}, doc="Line min (MW)")
m.PLmax = pyo.Param(m.L, initialize={l: float(PLmax[l]) for l in range(nl)}, doc="Line max (MW)")

# PTDFs (dimensionless)
m.MG = pyo.Param(m.L, m.G, initialize={
    (l, g): float(MG[l, g]) for l in range(nl) for g in range(ng)
}, doc="PTDF for generators")

m.MD = pyo.Param(m.L, m.D, initialize={
    (l, d): float(MD[l, d]) for l in range(nl) for d in range(nb)
}, doc="PTDF for demands")

# Demand (MW, mutable for future scenarios)
m.PD = pyo.Param(m.D, initialize={d: float(PD_MW[d]) for d in range(nb)}, 
                 mutable=True, doc="Demand at each bus (MW)")

# --- Variables ---
def pg_bounds_rule(m, g):
    return (m.PGmin[g], m.PGmax[g])

m.PG = pyo.Var(m.G, bounds=pg_bounds_rule, doc="Generator output (MW)")

# --- Objective: Quadratic generation cost ---
def obj_rule(m):
    return pyo.quicksum(
        m.c2[g] * m.PG[g]**2 + m.c1[g] * m.PG[g] + m.c0[g] 
        for g in m.G
    )

m.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize, doc="Total generation cost ($/hr)")

# --- Constraint 1: System balance (sum PG = sum PD) ---
def balance_rule(m):
    return pyo.quicksum(m.PG[g] for g in m.G) == pyo.quicksum(m.PD[d] for d in m.D)

m.Balance = pyo.Constraint(rule=balance_rule, doc="Power balance")

# --- Constraint 2: Line flow limits (PLmin <= M_G*PG + M_D*PD <= PLmax) ---
def line_flow_rule(m, l):
    flow = pyo.quicksum(m.MG[l, g] * m.PG[g] for g in m.G) + \
           pyo.quicksum(m.MD[l, d] * m.PD[d] for d in m.D)
    return pyo.inequality(m.PLmin[l], flow, m.PLmax[l])

m.LineFlow = pyo.Constraint(m.L, rule=line_flow_rule, doc="Line flow limits")

print("\nModel construction complete.")
print(f"  Variables: {len(list(m.G))} PG")
print(f"  Constraints: 1 balance + {len(list(m.L))} line limits = {1 + len(list(m.L))}")

# ============================================================================
# Solve with Gurobi
# ============================================================================
print("\n" + "="*70)
print("SOLVING WITH GUROBI")
print("="*70)

solver = pyo.SolverFactory('gurobi')
results = solver.solve(m, tee=True)

# ============================================================================
# Display Results
# ============================================================================
print("\n" + "="*70)
print("SOLUTION SUMMARY")
print("="*70)
print(f"Solver Status: {results.solver.status}")
print(f"Termination Condition: {results.solver.termination_condition}")

if results.solver.termination_condition == pyo.TerminationCondition.optimal:
    obj_val = pyo.value(m.OBJ)
    print(f"\nObjective Value: ${obj_val:,.2f}/hr")
    
    # Generator dispatch
    print(f"\n{'='*70}")
    print("GENERATOR DISPATCH")
    print(f"{'='*70}")
    print(f"{'Gen':<6} {'Pg (MW)':>12} {'Pmin':>10} {'Pmax':>10} {'c2':>12} {'c1':>12}")
    print("-" * 70)
    
    total_gen = 0.0
    active_count = 0
    
    for g in sorted(m.G):
        pg_val = pyo.value(m.PG[g])
        if pg_val > 1e-3:  # Show generators producing > 1 kW
            print(f"{g:<6} {pg_val:12.2f} {m.PGmin[g]:10.2f} {m.PGmax[g]:10.2f} "
                  f"{m.c2[g]:12.6f} {m.c1[g]:12.2f}")
            total_gen += pg_val
            active_count += 1
    
    print("-" * 70)
    print(f"Active Generators: {active_count}/{ng}")
    print(f"Total Generation: {total_gen:,.2f} MW")
    print(f"Total Demand: {sum(pyo.value(m.PD[d]) for d in m.D):,.2f} MW")
    print(f"Balance Error: {abs(total_gen - sum(pyo.value(m.PD[d]) for d in m.D)):.6f} MW")
    
    # Check line flows
    print(f"\n{'='*70}")
    print("LINE FLOW CHECK (first 10 lines)")
    print(f"{'='*70}")
    print(f"{'Line':<6} {'Flow (MW)':>12} {'Limit (MW)':>12} {'Utilization':>12}")
    print("-" * 70)
    
    for l in list(m.L)[:10]:
        flow = pyo.value(
            pyo.quicksum(m.MG[l, g] * m.PG[g] for g in m.G) +
            pyo.quicksum(m.MD[l, d] * m.PD[d] for d in m.D)
        )
        limit = m.PLmax[l]
        util = abs(flow / limit) * 100 if limit > 0 else 0
        print(f"{l:<6} {flow:12.2f} {limit:12.2f} {util:11.1f}%")
    
    print("="*70)
else:
    print(f"\n⚠ Solver did not find optimal solution!")
    print(f"Termination: {results.solver.termination_condition}")

Model built. Vars PG: 54 | Lines: 186 | Buses/D: 118
skipping further warning of this kind
Read LP format model from file C:\Users\63091\AppData\Local\Temp\tmpqdodfdi6.pyomo.lp
Reading time = 0.02 seconds
x1: 373 rows, 54 columns, 17636 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 373 rows, 54 columns and 17636 nonzeros
Model fingerprint: 0xd57dd34c
Model has 54 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-07, 1e+00]
  Objective range  [2e+01, 4e+01]
  QObjective range [2e-02, 5e+00]
  Bounds range     [1e+02, 8e+02]
  RHS range        [4e+01, 1e+04]
Presolve removed 372 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 54 columns, 54 nonzeros
Presolved model has 54 quadratic objective terms
Ordering time: 0.00s
skipping f